<a href="https://colab.research.google.com/github/LuizPitaAlmeida/imagecaptioningwithtransformers/blob/main/LuizPita_Aula_3_Avan%C3%A7ado_Image_Captioning_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Aula 3 - Avançado - Image Captioning - Template

In [1]:
TRAIN_MODE=False

In [2]:
!nvidia-smi

Tue Oct 13 23:58:55 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Download do dataset MS COCO reduzido para 23k exemplos:

In [3]:
!gsutil -m cp -n gs://neuralresearcher_data/unicamp/ia376j_2020s2/aula3/*_IMAGES_coco_5_cap_per_img.hdf5 .
!gsutil -m cp -n gs://neuralresearcher_data/unicamp/ia376j_2020s2/aula3/*_CAPTIONS_coco_5_cap_per_img.json .

Copying gs://neuralresearcher_data/unicamp/ia376j_2020s2/aula3/TEST_IMAGES_coco_5_cap_per_img.hdf5...
Copying gs://neuralresearcher_data/unicamp/ia376j_2020s2/aula3/TRAIN_IMAGES_coco_5_cap_per_img.hdf5...
Copying gs://neuralresearcher_data/unicamp/ia376j_2020s2/aula3/VAL_IMAGES_coco_5_cap_per_img.hdf5...
| [3/3 files][  4.7 GiB/  4.7 GiB] 100% Done  39.1 MiB/s ETA 00:00:00           
Operation completed over 3 objects/4.7 GiB.                                      
Copying gs://neuralresearcher_data/unicamp/ia376j_2020s2/aula3/TEST_CAPTIONS_coco_5_cap_per_img.json...
Copying gs://neuralresearcher_data/unicamp/ia376j_2020s2/aula3/TRAIN_CAPTIONS_coco_5_cap_per_img.json...
Copying gs://neuralresearcher_data/unicamp/ia376j_2020s2/aula3/VAL_CAPTIONS_coco_5_cap_per_img.json...
/ [3/3 files][  6.8 MiB/  6.8 MiB] 100% Done                                    
Operation completed over 3 objects/6.8 MiB.                                      


O dataset deve conter 23520, 1052 e 1047 imagens de treino, validação e teste.

Crie um modelo de image captioning completo similar ao do paper Show, Attend and Tell mas  usando a EfficientNet (pré-treinada) como codificador e usando o decodificador do T5, BART ou Pegasus. Para o decodificador, sugerimos utilizar a biblioteca da Hugginface e adaptar o decodificador para ter como entrada o feature map extraído da imagem.

Sugerimos ler a página abaixo para se familizarem como a biblioteca implementa a arquitetura encoder-decoder: https://huggingface.co/transformers/model_doc/encoderdecoder.html

## Setup do Hardware e do ambiente

In [4]:
!pip install --quiet transformers
!pip install --quiet pytorch-lightning
!pip install --quiet efficientnet_pytorch
!pip install --quiet sacrebleu

     |████████████████████████████████| 1.1MB 5.4MB/s 
     |████████████████████████████████| 3.0MB 35.2MB/s 
     |████████████████████████████████| 1.1MB 52.9MB/s 
     |████████████████████████████████| 890kB 63.0MB/s 
     |████████████████████████████████| 512kB 5.5MB/s 
     |████████████████████████████████| 276kB 24.4MB/s 
     |████████████████████████████████| 829kB 12.7MB/s 
     |████████████████████████████████| 92kB 11.7MB/s 
     |████████████████████████████████| 71kB 3.3MB/s 


In [5]:
import os
import random
import json
from collections import Counter, OrderedDict
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
import h5py as h5

import nltk
import torch
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import torch.nn as nn
from torch.utils.data import DataLoader

from efficientnet_pytorch import EfficientNet

from torchsummary import summary
from transformers import EncoderDecoderModel, AutoTokenizer

import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint

import sacrebleu

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
seed = 0
random.seed(seed)
torch.random.manual_seed(seed)
torch.cuda.manual_seed(seed)

## Parameters

In [7]:
MAX_SEQUENCE_LEN = 49
EFFICIENTNET_MODEL = 'efficientnet-b0'
BERT_MODEL = 'bert-base-uncased'
IMAGE_SIZE = EfficientNet.get_image_size(EFFICIENTNET_MODEL)
DEFAULT_BATCH_SIZE = 10
TOKENIZER = AutoTokenizer.from_pretrained(BERT_MODEL)


In [8]:
if not TRAIN_MODE:
    testing = TOKENIZER.encode('i like pizza.')
    print(testing)

[101, 1045, 2066, 10733, 1012, 102]


In [9]:
if not TRAIN_MODE:
    print(TOKENIZER.decode(testing))

[CLS] i like pizza. [SEP]


## Classe de Vocabulário apartir dos dados de treino

In [ ]:
class Vocabulary(object):
    """Simple vocabulary wrapper."""
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0

    def add_word(self, word):
        if word not in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1

    def __call__(self, word):
        if word not in self.word2idx:
            return self.word2idx['<unk>']
        return self.word2idx[word]

    def __len__(self):
        return len(self.word2idx)


def build_vocab(json_path, min_freq_to_count_in_vocab):
    """Build a simple vocabulary wrapper."""
    counter = Counter()
    nltk.download('punkt')

    with open(json_path, 'r') as j_file:
        captions_file = json.load(j_file)
        captions = [caption for five_captions in captions_file
                    for caption in five_captions]

    for i, caption in enumerate(captions):
        tokens = nltk.tokenize.word_tokenize(caption.lower())
        counter.update(tokens)

    # If the word frequency is less than 'threshold', then the word is
    # discarded.
    words = [word for word, cnt in counter.items()
             if cnt >= min_freq_to_count_in_vocab]

    # Create a vocab wrapper and add some special tokens.
    vocab = Vocabulary()
    vocab.add_word('<pad>')
    vocab.add_word('<start>')
    vocab.add_word('<end>')
    vocab.add_word('<unk>')

    # Add the words to the vocabulary.efficient_net
    for i, word in enumerate(words):
        vocab.add_word(word)
    return vocab

vocab = build_vocab(
    'TRAIN_CAPTIONS_coco_5_cap_per_img.json',
    min_freq_to_count_in_vocab=4
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#dict(list(vocab.word2idx.items())[:20])

# Leitura das imagens e captions

Aqui também é realizado a tokenização e a conversão de tokens para ids

In [ ]:
class HDF5ImageDataset(Dataset):
    """Classe para leitura das imagens"""

    def __init__(self, h5_path, h5_key=None, preprocess=False,
                 img_size=IMAGE_SIZE):
        """Variáveis e métodos criados ao instanciar um objeto"""
        # Uso da lib h5py para fazer carregar o arquivo hdf5 em mémoria
        self.h5_file = h5.File(h5_path, 'r')
        
        # Os arquivos h5py são lidos em uma espécie de dicionário. Durante
        # sua criação o usuário definio o nome das chaves. O método a seguir
        # permite setar a chave que contém os arquivos de imagem. Caso nenhuma
        # chave seja passada o valor assumido é a primeira chave. Em nosso caso
        # só existe uma única chave de nome "images"
        self.set_key(h5_key)
        
        # Em Visão Computacional é comum aplicarmos alguns preprocessamentos a
        # imagem que servirá de entrada da rede. Como exemplo podemos citar
        # resize, crop, rodações, espelhamento e normalização. O pytorch possui
        # no pacote torchvision uma classe transform que facilita a execução
        # desses pré-processamentos. Essa classe recebe como entrada imagens no
        # pacote PIL. Em nosso caso, definimos nossa transformação como sendo a
        # conversão da imagem em um tensor do pytorch.
        self.preprocess = preprocess 
        self.transform = transforms.Compose([
            transforms.Resize(img_size),
            transforms.CenterCrop(img_size),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            ),
        ])

    def __getitem__(self, index):
        """As classes de dataset do Pytorch utilizam esse método mágico para
        facilitar a iteração sobre datasets. Com ela podemos obter um elemento
        do dataset realizando uma simples indexação. 
        """
        # Leitura da imagem no índice indicado
        image = self.h5_file[self.key][index]

        # Verificação do shape da imagem. Nesse tutorial assumimos que as imagem
        # possui 3 canais e é lida no formato [channels, rows, cols]. Esse
        # método checa o shape e a dimensão das imagens e, quando no formato a
        # cima modifica para o formato [rows, cols, channels]     
        image = self._check_image(image)
        
        # Ao ser lida pelo pacote h5py a imagem vem em formato numpy, aqui
        # convertemos para PIL image de forma que poderemos usar a
        # transformação do torchvision.
        image = Image.fromarray(image) 

        # Aplicação da transformação do pyvision. No exemplo que estamos
        # trabalhando, esse processo poderia ser feito de forma simplificada,
        # convertendo diretamente o numpy array gerado pelo h5py em um tensor
        # torch. Optamos por mostrar a nível de aprendizado a classe transform
        if self.preprocess:
            image = self.transform(image)
        return image
  
    def __len__(self):
        """Método mágico usado para informa facilmente o número de elementos
        presentes no dataset
        """
        return len(self.h5_file[self.key])

    def _check_image(self, image):
        """Verificação e ajuste nas dimensões e shapes da imagem"""
        # Verifica se a imagem possui apenas três dimensões
        if image.ndim == 3:
            # Verifica se a imagem está no formato [channels, rows, cols]
            # e inverte para o formato [rows, cols, channels] 
            if image.shape[0] == 3:
                return np.moveaxis(image, 0, -1)
            
            # Verifica se a imagem já se encontra no formato
            # [rows, cols, channels], se sim passa adiante 
            if image.shape[-1] == 3:
                return image
            
            # Casos que fogem os padrões acima são tratados como erro
            print('Image does not have 3 channels.')
            raise ValueError
        
        # Caso possua menos ou mais dimensões, um erro é apontado
        else:
            print('Image dimension is not 3.')
            raise ValueError
            
    def set_key(self, h5_key):
        """Define a chave do h5py que será usada para carregar as imagens"""
        if h5_key: 
            self.key = h5_key
        else:
            self.key = list(self.h5_file[0])

    def h5_info(self):
        """Imprime as chaves que compõem o arquivo de carregado pelo h5py"""
        print('Keys:', list(self.h5_file))

    def plot(self, index):
        image = self.h5_file[self.key][index]      
        image = self._check_image(image)
        image = Image.fromarray(image)
        plt.imshow(image)
        plt.xticks([])
        plt.yticks([])

In [ ]:
if not TRAIN_MODE:    
    class MSCOCO_Captions__Dataset_Grouped(Dataset):
        def __init__(self, h5_path, json_path, h5_key=None,
                    preprocess=True):
            with open(json_path, 'r') as annotation_file:
                self.captions = json.load(annotation_file)
                #self.captions = [caption for five_captions in captions_file
                #                 for caption in five_captions]
            # Utilizamos a classe anteriormente criada para carregar os arquivos de
            # imagens
            self.images = HDF5ImageDataset(h5_path, h5_key, preprocess)
            self.max_length = MAX_SEQUENCE_LEN

        def __getitem__(self, index):
            text = self.captions[index]
            encoded = self._data_encoder(text)

            token_ids = torch.tensor(encoded['input_ids']).type(torch.long)
            attention_mask = torch.tensor(encoded['attention_mask']).type(torch.long)
            token_type_ids = torch.tensor(encoded['token_type_ids']).type(torch.long)
            
            return self.images[index], token_ids, attention_mask, token_type_ids
            
            #caption_as_ids = [
                #self.captions2ids(nltk.tokenize.word_tokenize(
                #    caption.lower()
                #)
                #) for caption in self.captions[index]
            #]
            #return self.images[index], torch.LongTensor(caption_as_ids) #int(index/5)
        
        def __len__(self):
            return len(self.captions)

        def _data_encoder(self, text):
            return TOKENIZER.encode_plus(
                text,
                max_length = self.max_length,
                padding='max_length',
                return_attention_mask=True)

        def captions2ids(self, seq):
            padded_seq = (
                [self.vocab(word) for word in seq] + self.maxseqlen*[self.pad_id]
            )
            return padded_seq[:self.maxseqlen]

        def ids2caption(self, ids_seqs):
            return [[self.vocab.idx2word[id] for id in ids_seq]
                    for ids_seq in ids_seqs]

        def plot(self, index):
            self.images.plot(index) #int(index/5)
            plt.text(-1, -1, '\n'.join(self.captions[index]))

In [ ]:
class MSCOCO_Captions__Dataset(Dataset):
    def __init__(self, h5_path, json_path, vocab, h5_key=None,
                 preprocess=True):
        with open(json_path, 'r') as annotation_file:
            captions_file = json.load(annotation_file)
            self.captions = [caption for five_captions in captions_file
                             for caption in five_captions]
        # Utilizamos a classe anteriormente criada para carregar os arquivos de
        # imagens
        self.images = HDF5ImageDataset(h5_path, h5_key, preprocess)
        self.vocab = vocab
        self.max_length = MAX_SEQUENCE_LEN
        self.pad_id = vocab('<pad>')

    def __getitem__(self, index):
        text = self.captions[index]
        encoded = self._data_encoder(text)

        token_ids = torch.tensor(encoded['input_ids']).type(torch.long)
        attention_mask = torch.tensor(encoded['attention_mask']).type(torch.long)
        #token_type_ids = torch.tensor(encoded['token_type_ids']).type(torch.long)
        
        return self.images[index], token_ids, attention_mask #, token_type_ids 
        
        #caption_as_ids = [
        #    self.vocab(word) for word in
        #    TOKENIZER.encode(self.captions[index].lower()) #nltk.tokenize.word_tokenize
        #]
        #padded_seq = caption_as_ids + self.maxseqlen*[self.pad_id]
        #return self.images[int(index/5)], torch.LongTensor(padded_seq[:self.maxseqlen])
    
    def __len__(self):
        return len(self.captions)

    def _data_encoder(self, text):
        return TOKENIZER.encode_plus(
            text,
            max_length = self.max_length,
            padding='max_length',
            return_attention_mask=True)

    def ids2caption(self, ids_seq):
        return [self.vocab.idx2word[id] for id in ids_seq]

    def plot(self, index):
        self.images.plot(int(index/5))
        plt.title(self.captions[index])

In [ ]:
if not TRAIN_MODE:
    # Validando a classe
    test = MSCOCO_Captions__Dataset(
        'TEST_IMAGES_coco_5_cap_per_img.hdf5',
        'TEST_CAPTIONS_coco_5_cap_per_img.json',
        vocab = vocab,
        h5_key='images',
        preprocess=True
    )

    # Printando e plotando o primeito elemento
    element_id = 0
    print(test.captions[0])
    image, caption_ids, attention_mask, token_type_ids = test[element_id]
    print('Image Tensor (tipo, shape, data):', type(image), image.shape)
    print(image)
    print('\nCaption Ids: (tipo, shape, data):', type(caption_ids), caption_ids.shape)
    print(caption_ids)
    #print(test.ids2caption(caption_ids.tolist())
    #for tokens in test.ids2caption(caption_ids.tolist()):
    #    print(tokens)
    print([TOKENIZER.ids_to_tokens[tokens_ids.item()] for tokens_ids in caption_ids])

    print('\n####### Plot ########')
    test.plot(element_id)

### Criando os dataset

In [ ]:
train_dataset = MSCOCO_Captions__Dataset(
    'TRAIN_IMAGES_coco_5_cap_per_img.hdf5',
    'TRAIN_CAPTIONS_coco_5_cap_per_img.json',
    vocab=vocab,
    h5_key='images'
)

valid_dataset = MSCOCO_Captions__Dataset(
    'VAL_IMAGES_coco_5_cap_per_img.hdf5',
    'VAL_CAPTIONS_coco_5_cap_per_img.json',
    vocab=vocab,
    h5_key='images'
)

test_dataset = MSCOCO_Captions__Dataset(
    'TEST_IMAGES_coco_5_cap_per_img.hdf5',
    'TEST_CAPTIONS_coco_5_cap_per_img.json',
    vocab=vocab,
    h5_key='images'
)

print('Length of train_dataset:', len(train_dataset))
print('Length of valid_dataset:', len(valid_dataset))
print('Length of test_dataset:', len(test_dataset))

Length of train_dataset: 117600
Length of valid_dataset: 5260
Length of test_dataset: 5235


In [ ]:
if not TRAIN_MODE:
    with open('TRAIN_CAPTIONS_coco_5_cap_per_img.json', 'r') as annotation_file:
        ann = json.load(annotation_file)
    lengths = [len(caption) for five_captions in ann for caption in five_captions]
    print('Max sequence length:', max(lengths))

    from matplotlib import pyplot as plt
    plt.hist(lengths)

# Dataloaders



In [ ]:
if not TRAIN_MODE:
    BATCH_SIZE = 5

    train_dataloader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=1,
        drop_last=True,
    )

    batch = next(iter(train_dataloader))

    print('Tamanho do batch para as imagens:', batch[0].shape)
    print('Tamanho do batch para os captions:', batch[1].shape)
    print()
    print(batch[1][1])
    print('Convertendo para caption ...')
    #print(test_dataset.ids2caption(batch[1][1].tolist()))
    #for tokens in test_dataset.ids2caption(batch[1][1].tolist()):
    #    print(tokens)
    print([TOKENIZER.ids_to_tokens[tokens_ids.item()]
        for tokens_ids in batch[1][1]])
    print(batch[2][1])

In [ ]:
if not TRAIN_MODE:
    train_dataloader = DataLoader(
        train_dataset,
        batch_size=DEFAULT_BATCH_SIZE,
        shuffle=False,
        num_workers=1,
        drop_last=True,
    )

    valid_dataloader = DataLoader(
        valid_dataset,
        batch_size=DEFAULT_BATCH_SIZE,
        shuffle=False,
        num_workers=1,
        drop_last=True,
    ) 

    test_dataloader = DataLoader(
        test_dataset,
        batch_size=DEFAULT_BATCH_SIZE,
        shuffle=False,
        num_workers=1,
        drop_last=True,
    )

## lightning datamodule

In [ ]:
class DataLoaderModule(pl.LightningDataModule):
    def __init__(self, train_database, valid_database, test_database):
        super().__init__()
        self.train_data = train_database
        self.valid_data = train_database
        self.test_data = train_database

    def train_dataloader(self):
        return DataLoader(
            self.train_data,
            batch_size=DEFAULT_BATCH_SIZE,
            shuffle=False,
            num_workers=1,
            drop_last=False,
        )

    def val_dataloader(self):
        return DataLoader(
            self.valid_data,
            batch_size=DEFAULT_BATCH_SIZE,
            shuffle=False,
            num_workers=1,
            drop_last=False,
        )

    def test_dataloader(self):
        return DataLoader(
            self.test_data,
            batch_size=DEFAULT_BATCH_SIZE,
            shuffle=False,
            num_workers=1,
            drop_last=False,
        )

DATALOADER_MD = DataLoaderModule(
    train_database=train_dataset,
    valid_database=valid_dataset,
    test_database=test_dataset
)

#Image Encoder

Encoder using Efficient Net

In [ ]:
# MAX_SEQUENCE_LEN = 49
class ImageEncoder(nn.Module):
    def __init__(self): #finetune=False
        super().__init__()
        self.embedding_dim = 768 # Conflito com os embbedings posicionais 
        self.cnn = EfficientNet.from_pretrained(EFFICIENTNET_MODEL)
        
        # Freeze model weights to not train
        for param in self.cnn.parameters():
            param.requires_grad = False

        self.encoding_layers = nn.Sequential(OrderedDict([ 
           ('EmbeddingDim_Conv2d', nn.Conv2d(1280, self.embedding_dim, 3, 1, 1)),
           ('Final_DropOut', nn.Dropout(p=0.1))
        ]))

        # If fine-tuning, only fine-tunes convolutional blocks 2 through 4
        #if finetune:
        #    for convolutional_layer in list(self.cnn_model.children())[5:]:
        #        for param in convolutional_layer.parameters():
        #            param.requires_grad = True

    def forward(self, x):
        x = self.cnn.extract_features(x)
        x = self.encoding_layers(x)
        x = x.reshape(-1, self.embedding_dim, MAX_SEQUENCE_LEN)
        x = x.transpose(1, 2)
        return x

In [ ]:
if not TRAIN_MODE:
    test_model = ImageEncoder()
    test_model.to(device)
    summary(test_model, input_size=(3, IMAGE_SIZE, IMAGE_SIZE))

# Transformer Encoder Decoder Model

Vamos iniciar nossos testes utilizando o BERT

In [ ]:
if not TRAIN_MODE:
    bert_model = EncoderDecoderModel.from_encoder_decoder_pretrained(
        BERT_MODEL, BERT_MODEL
    )

In [ ]:
if not TRAIN_MODE:
    batch = next(iter(train_dataloader))
    input_test = batch[0].to(device)
    decoder_input_test = batch[1].to(device)
    decoder_input_mask = batch[2].to(device)
    encoder_embedding = test_model(input_test)
    test_model.to('cpu')
    del test_model

In [ ]:
#print(decoder_input_test.shape)

In [ ]:
if not TRAIN_MODE:
    bert_model.to(device)
    outputs = bert_model(
        inputs_embeds=encoder_embedding, decoder_input_ids=decoder_input_test, 
        decoder_attention_mask=decoder_input_mask, return_dict=True
    )

In [ ]:
if not TRAIN_MODE:
    out = outputs['logits'].permute(0,2,1) 
    #decoder_input_test[0].shape, out[0].shape
    #nn.CrossEntropyLoss()(out, decoder_input_test)

In [ ]:
#torch.argmax(outputs['logits'][2], dim=-1)

In [ ]:
#test_tensor = torch.Tensor([1012, 1012, 1012, 1012, 1012, 1012, 1012, 1012, 1012, 1012, 1012, 1012,
#        1012, 1012, 1012, 1012, 1012, 1012, 1012, 1012, 1012, 1012, 1012, 1012,
#        1012, 1012, 1012, 1012, 1012, 1012, 1012, 1012, 1012, 1012, 1012, 1012,
 #       1012, 1012, 1012, 1012, 1012, 1012, 1012, 1012, 1012, 1012, 1012, 1012,
 #       1012])
#TOKENIZER.decode(test_tensor)

In [ ]:
if not TRAIN_MODE:
    bert_model.to('cpu')
    del bert_model
    torch.cuda.empty_cache()

# testing sacrebleu

In [ ]:
if not TRAIN_MODE:
    refs = [
            ['The dog bit the man.', 'It was not unexpected.', 'The man bit him first.'],
            ['The dog had bit the man.', 'No one was surprised.', 'The man had bitten the dog.']
    ]
    sys = ['The dog bit the man.', "It wasn't surprising.", 'The man had just bitten him.']
    bleu = sacrebleu.corpus_bleu(sys, refs)
    print(bleu.score)

In [ ]:
#y_target = [2000, 2002, 1995]
#TOKENIZER.decode(y_target)

In [ ]:
if not TRAIN_MODE:
    out_test = [
                {"pred_seq": 'i like dalmatas.', "target_seq": 'i hate dalmatas'},
                {"pred_seq": 'i like chow chow.', "target_seq": 'i hate chow chow'},
                {"pred_seq": 'i like shitsu.', "target_seq": 'i hate shitsu'},
                {"pred_seq": 'i like pastor.', "target_seq": 'i hate pastor'},
                {"pred_seq": 'i like poodle.', "target_seq": 'i hate poodle'},

                {"pred_seq": 'i like pizza.', "target_seq": 'i hate pizza'},
                {"pred_seq": 'i like hotdog.', "target_seq": 'i hate hotdog'},
                {"pred_seq": 'i like burguer.', "target_seq": 'i hate burguer'},
                {"pred_seq": 'i like apple.', "target_seq": 'i hate apple'},
                {"pred_seq": 'i like banana.', "target_seq": 'i hate banana'},

                {"pred_seq": 'i like soccer.', "target_seq": 'i hate soccer'},
                {"pred_seq": 'i like volley.', "target_seq": 'i hate volley'},
                {"pred_seq": 'i like tennis.', "target_seq": 'i hate tennis'},
                {"pred_seq": 'i like video games.', "target_seq": 'i video games'},
                {"pred_seq": 'i like movies.', "target_seq": 'i hate movies'}
    ]

In [ ]:
if not TRAIN_MODE:
    predictions = [x["pred_seq"] for x in out_test]
    all_targets = [x["target_seq"] for x in out_test]
    print(predictions)
    print(all_targets)

In [ ]:
#grouped_targets = [all_targets[start::5] for start in range(5)]
#grouped_targets

In [ ]:
#grouped_predictions = [predictions[start::5] for start in range(5)]

In [ ]:
#len(predictions[::5]), len(grouped_targets), len(grouped_targets[0])

In [ ]:
#bleu = sacrebleu.corpus_bleu(predictions[::5], grouped_targets)
#bleu.score

#modelo completo

In [ ]:
class ImageCaptioning(pl.LightningModule):
    """Encoder-Decoder Image Caption Model"""
    def __init__(self):
        super().__init__()

        self.encoder = ImageEncoder()
        self.decoder = EncoderDecoderModel.from_encoder_decoder_pretrained(
            BERT_MODEL, BERT_MODEL
        )
        self.loss = torch.nn.CrossEntropyLoss()
    
    def forward(self, x):
        images, captions_ids, captions_len_mask  = x
        encoder_embedding = self.encoder(images)
        outputs = self.decoder(
            inputs_embeds=encoder_embedding,
            decoder_input_ids=captions_ids, 
            decoder_attention_mask=captions_len_mask,
            return_dict=True
        )
        return outputs['logits']
    
    def training_step(self, x, batch_idx):
        images, captions_ids, captions_len_mask  = x
        logits = self(x)
        #y_target = captions_ids[captions_len_mask == 1]
        #loss = self.loss(logits, y_target)
        loss = self.loss(logits.permute(0,2,1), captions_ids)

        return {"loss": loss}
    
    def validation_step(self, x, batch_idx):
        images, captions_ids, captions_len_mask  = x
        logits = self(x)
        #y_target = captions_ids[captions_len_mask == 1]
        #loss = self.loss(logits, y_target)
        loss = self.loss(logits.permute(0,2,1), captions_ids)

        pred_seq = [TOKENIZER.decode(torch.argmax(sample, dim=-1))
                    for sample in logits]
        target_seq = [TOKENIZER.decode(target) for target in captions_ids]

        return {"val_loss": loss,
                "pred_seq": pred_seq,
                "target_seq": target_seq}
    
    def test_step(self, x, batch_idx):
        images, captions_ids, captions_len_mask  = x
        logits = self(x)
        #y_target = captions_ids[captions_len_mask == 1]
        #loss = self.loss(logits, y_target)
        loss = self.loss(logits.permute(0,2,1), captions_ids)

        #pred_seq = TOKENIZER.decode(torch.argmax(logits, dim=-1))
        #target_seq = TOKENIZER.decode(y_target)
        pred_seq = [TOKENIZER.decode(torch.argmax(sample, dim=-1))
                    for sample in logits]
        # não usar tokenizer aqui, dá um jeito de pegar o original
        # Eliminar o pad do score
        target_seq = [TOKENIZER.decode(target) for target in captions_ids]

        return {"test_loss": loss,
                "test_pred_seq": pred_seq,
                "test_target_seq": target_seq}
    
    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([x["loss"] for x in outputs]).mean()

        self.log("avg_loss", avg_loss, prog_bar=True)


    def validation_epoch_end(self, outputs):
        # Metric 1
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()

        # Metric 2
        predictions = outputs[0]["pred_seq"][::5]
        targets = outputs[0]["target_seq"]
        grouped_targets = [targets[start::5] for start in range(5)]

        bleu = sacrebleu.corpus_bleu(predictions, grouped_targets)
                
        self.log("valid_loss", avg_loss, prog_bar=True)
        self.log("bleu", bleu.score, prog_bar=True)
        #self.log("bleu1234", bleu.precisions, prog_bar=True)

        print('Pred:', predictions[0])
        print('Targets:', grouped_targets[0])

        return {"val_loss": avg_loss, "bleu": bleu.score,
                "bleu1": bleu.precisions[0], "bleu2": bleu.precisions[1],
                "bleu3": bleu.precisions[2], "bleu4": bleu.precisions[3]
                }

    def test_epoch_end(self, outputs):
        # Metric 1
        avg_loss = torch.stack([x["test_loss"] for x in outputs]).mean()

        # Metric 2
        predictions = [x["test_pred_seq"] for x in outputs][::5]
        targets = [x["test_target_seq"] for x in outputs]
        grouped_targets = [targets[start::5] for start in range(5)]

        bleu = sacrebleu.corpus_bleu(predictions, grouped_targets)
                    
        self.log("test_loss", avg_loss, prog_bar=True)
        self.log("test_bleu", bleu.score, prog_bar=True)
        #self.log("bleu1234", bleu.precisions, prog_bar=True)

        print('Pred:', predictions[0])
        print('Targets:', grouped_targets[0])

        return {"test_loss": avg_loss, "bleu": bleu.score,
                "bleu1": bleu.precisions[0], "bleu2": bleu.precisions[1],
                "bleu3": bleu.precisions[2], "bleu4": bleu.precisions[3]
                }

            
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())


In [ ]:
def setup_logs_and_callbacks(output_path, exp_name):
    tensorboard_path = os.path.join(output_path, "logs")
    logs_folder = os.path.join(tensorboard_path, exp_name)
    os.makedirs(logs_folder, exist_ok=True)
    ckpt_path = os.path.join(logs_folder, "-{epoch}")

    logger = TensorBoardLogger(tensorboard_path, exp_name)
    checkpoint_callback = ModelCheckpoint(prefix=exp_name,
                                          filepath=ckpt_path,
                                          monitor='val_loss',
                                          mode="min")
    return logger, checkpoint_callback

# testando o treinamento

In [ ]:
model = ImageCaptioning()
trainer = pl.Trainer(fast_dev_run=True, gpus=1)
trainer.fit(model, datamodule=DATALOADER_MD)

Loaded pretrained weights for efficientnet-b0


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.key.bias', 'bert.encoder.layer

Pred: midfield midfield midfield midfield midfield midfield midfield midfield midfield midfield indicative midfield midfield midfield midfield midfield midfield midfield midfield midfield midfield midfield midfield midfield midfield midfield midfield midfield midfield midfield midfield midfield midfield midfield midfield midfield midfield midfield midfield midfield midfield midfield midfield midfield midfield midfield midfield midfield midfield
Targets: ['[CLS] a woman in a room with a cat. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]', '[CLS] a group of people riding on the back of a loaded red pickup truck. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The validation_epoch_end should not return anything as of 9.1.to log, use self.log(...) or self.write(...) directly in the LightningModule
  warnings.warn(*args, **kwargs)


1

# treinamento

In [ ]:
logger, checkpoint_callback = setup_logs_and_callbacks('/content/testing', 'testing')

In [ ]:
#trainer = Trainer(
#    resume_from_checkpoint=None,
#    gpus=1,
#    checkpoint_callback=checkpoint_callback,
#    early_stop_callback=False,
#    logger=logger,
#    accumulate_grad_batches=ACC_GRAD_BATCHES,
#    max_epochs=MAX_EPOCHS,
    #val_check_interval=VALIDATION_CHECK,
#    progress_bar_refresh_rate=PROGESS_BAR_RATE,
#)